In [25]:
from bs4 import BeautifulSoup, Comment
import pandas as pd
import re
import requests
from selenium import webdriver
import time
from tqdm import tqdm

In [70]:
geckodriver_path = r'C:\Users\nicol\anaconda3\Library\bin\geckodriver'
browser = webdriver.Firefox(executable_path=geckodriver_path)

In [ ]:
import os

competitors = pd.read_excel(f'{os.getcwd()}\\Libro1.xlsx', engine="openpyxl", index_col=0)
                            #usecols=['ID', 'Relación', 'Tópicos', 'Nombre', 'Comuna', 'Cuenta con clases online',
                            #         'Cuenta con clases presenciales', 'Instagram', 'has_instagram',
                            #         'Publicaciones', 'Norm Publicaciones', 'Seguidores Instagram',
                            #         'Norm Seguidores', 'Seguidos', 'Página web'])

display(competitors)

<h2>Instagram</h2>

In [ ]:
for row in tqdm(range(competitors.shape[0])):
    new_cols = ['Publicaciones', 'Seguidores', 'Seguidos']
    url = competitors.at[row, 'Instagram']
    
    try:
        browser.get(url)
        time.sleep(5)

        html_instagram = browser.page_source
        soup_instagram = BeautifulSoup(html_instagram, 'lxml')

        elements = soup_instagram.find_all('a', class_ = '-nal3')
        
        for i, element in enumerate(elements):
            re_element = re.search('([0-9]\,[0-9]|[0-9]\.[0-9]]+)', element.text)#.group(1)
            raw_number = re_element.group(1)
            
            number_commas = raw_number.replace(',', '')
            clean_number = raw_number.replace('.', '')
            
            
            
            try:
                multiplier = re_element.group(2)
                number = clean_number * 1000
            
            except:
                number = clean_number              
                
            try:
                competitors.at[row, new_cols[i]] = number.text
                
            except:
                competitors[new_cols[i]] = None
                competitors.at[row, new_cols[i]] = number.text
            
    except Exception as e:
        for col in new_cols:
            competitors.at[row, col] = None
    
    print(re_element)        
    break

In [ ]:
display(competitors)#.to_excel('cacapoto.xlsx')

<h2>Facebook</h2>

In [ ]:
for row in tqdm(range(competitors.shape[0])):
    new_cols = ['Seguidores Instagram', 'Me gusta']
    url = competitors.at[row, 'Facebook']
    
    try:
        html_facebook = requests.get(url)
        soup_facebook = BeautifulSoup(html_facebook.text, 'lxml')

        seguidores = re.search('([0-9\.]+) personas siguen esto', html_facebook.text).group(1)
        me_gusta = re.search('A ([0-9\.]+) personas les gusta esto', html_facebook.text).group(1)
        #estuvieron = re.search('([0-9\.]+) personas estuvieron aquí', html_facebook.text).group(1)

        #print(f'Seguidores: {seguidores}'.replace('.', ''), '\n'
        #      f'Me gusta: {me_gusta}'.replace('.', ''), '\n')#,
              #f'Asistentes: {estuvieron}'.replace('.', ''), '\n')

        try:
            competitors.at[row, 'Seguidores Instagram'] = seguidores.replace('.', '')
            competitors.at[row, 'Me gusta'] = me_gusta.replace('.', '')

        except:
            competitors[new_cols[i]] = None
            competitors.at[row, 'Seguidores Instagram'] = seguidores.replace('.', '')
            competitors.at[row, 'Me gusta'] = me_gusta.replace('.', '')
            
    except Exception as e:
        for col in new_cols:
            competitors.at[row, col] = None

In [ ]:
competitors.to_excel('cacapoto.xlsx')

<h2>Youtube</h2>

In [ ]:
for row in tqdm(range(competitors.shape[0])):
    new_cols = ['Suscriptores']
    url = competitors.at[row, 'Youtube']

    try:
        html_youtube = requests.get(url)
        #soup_youtube = BeautifulSoup(html_youtube.text, 'lxml')

        subscribers = re.search('([0-9\.]+) suscriptores', html_youtube.text).group(1)
        #print(f'Suscriptores: {subscribers}')
    
        try:
            competitors.at[row, 'Suscriptores'] = subscribers.replace('.', '')

        except:
            competitors[new_cols[i]] = None
            competitors.at[row, 'Suscriptores'] = subscribers.replace('.', '')
            
    except Exception as e:
        for col in new_cols:
            competitors.at[row, 'Suscriptores'] = None

In [ ]:
competitors.to_excel('cacapoto.xlsx')

<h2>Twitter</h2>

In [ ]:
#for row in tqdm(range(competitors.shape[0])):
#    new_cols = ['Suscriptores']
#    url = competitors.at[row, 'Twitter']
    
browser.get('https://twitter.com/Danaamar')
time.sleep(5)

html_twitter = browser.page_source
soup_twitter = BeautifulSoup(html_twitter, 'lxml')

#following = re.search('([0-9\.]+) personas siguen esto', html_twitter).group(1)
#followers = re.search('([0-9\.]+) personas siguen esto', html_twitter).group(1)

#print(following, followers, '\n\n', soup_twitter.prettify())

In [ ]:
soup_twitter.select('//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div[1]/div/div[2]/div/div/div[1]/div/div[5]/div[1]/a/span[1]/span')

<h2>Webpage</h2>

In [108]:
def get_text(basic_url, browser):
    #html_webpage = requests.get(basic_url)
    browser.get(basic_url)
    time.sleep(3)
    
    html_webpage = browser.page_source
    soup_webpage = BeautifulSoup(html_webpage, 'lxml') #.content, 'lxml')

    raw_links = set([re.search(f'{basic_url}(.*)', f'{basic_url}{tag.get("href")}').group(1)
                     for tag in soup_webpage.find_all('a')] + [basic_url])
    
    forbidden = ['facebook', 'youtube', 'vimeo', 'instagram', 'outlook', 'whatsapp', 'twitter', 'linkedin']
    links = []
    
    for link in raw_links:
        check = 0

        for page in forbidden:

            try:
                re_test = re.search(f'({page})', link).group(1)
                check += 1

            except:
                pass

            try:
                file_name = re.search('(https://www\.|http://www\.|https://|http://)(.*)\.', link).group(2)

            except:
                check += 1

        if check == 0:
            links.append(link)
    
    set_links = set(links)

    dict_texts = {}
    dict_clean = {'á': 'a', 'é': 'e','í': 'i', 'ó': 'o', 'ú':'u', '.': '', ',': ' '}
    prohibited = ['facebook', 'twitter', 'youtube', 'redes', 'sociales', 'videos',
                  'home', 'contacto', 'noticias', 'navegacion', 'enlace']

    for link in set_links:
        list_text = []

        try:
            #html_link = requests.get(link)
            browser.get(link)
            time.sleep(3)
            
            html_link = browser.page_source
            soup_link = BeautifulSoup(html_link, 'lxml') #.content, 'lxml')

            for text in soup_link.body.find_all(string=True):
                if text.parent.name not in ['script', 'meta', 'link', 'style'] and not isinstance(text, Comment) and text != '\n':
                    list_text.append(text.strip())

            raw_text = ','.join(list_text).lower()
            clean_chars = ''.join([dict_clean[char] if char in dict_clean.keys() else char for char in raw_text ])
            clean_words = ' '.join([word for word in clean_chars.split(' ') if word not in prohibited])
            #print([word for word in clean_chars if word not in prohibited])

            dict_texts[link] = clean_words

        except Exception as e:
            pass

    text = ', '.join(list(dict_texts.values()))
    
    return text

In [116]:
### counter = 0
dict_texts = {}

for i, row in enumerate(competitors.index):
    url = competitors.at[row, 'Página web']
    
    if isinstance(url, str):
        basic_url = re.search('(.*\.com|.*\.cl)', url).group(1)
        file_name = re.search('(https://www\.|http://www\.|https://|http://)(.*)\.', basic_url).group(2)
        text_in_url = get_text(basic_url, browser)
        
        dict_texts[file_name] = (len(text_in_url), text_in_url, basic_url)
        print(counter, basic_url, file_name, len(text_in_url))
        counter += 1


103 https://www.bellydancer.cl bellydancer 3864
104 https://www.safidanzaarabe.com safidanzaarabe 10512
105 https://www.bamboleo.cl bamboleo 2567
106 https://lamorenetastudio.com lamorenetastudio 84116
107 https://nataliadufuur.com nataliadufuur 128081
108 https://www.artfitcenter.cl artfitcenter 42158
109 https://danielabellydancer.com danielabellydancer 83147
110 https://danson.cl danson 38946
111 https://poledancetraining.cl poledancetraining 34333
112 https://www.sandricastillo.com sandricastillo 9616
113 http://www.danaamar.com danaamar 36927
114 http://www.seyyalbellydancer.com seyyalbellydancer 143333
115 http://www.nayra.cl nayra 0
116 https://danzadelvientretribal.cl danzadelvientretribal 18832
117 https://www.arabesca.cl arabesca 36753
118 https://www.bailesvalero.cl bailesvalero 137218
119 https://escueladeburlesquesantiago.cl escueladeburlesquesantiago 28454
120 https://rumbachilena.cl rumbachilena 194527
121 http://www.nurdeoriente.cl nurdeoriente 688
122 https://www.eupho

In [179]:
all_text = ''.join([value[1] for value in dict_texts.values()])
list_words = all_text.split(' ')

df_test = pd.DataFrame({'words': list_words})
df_uniques = pd.unique(df_test['words'])

df_test.to_excel('test.xlsx')

In [182]:
df_test = pd.read_excel('test.xlsx', engine="openpyxl")
display(df_test.tail(3))

,Unnamed: 0,words,repetitions,lenght
440787,438088,-70580343,1,9
440788,438116,4792,1,4
440789,438148,hcw9+8v,1,7


In [184]:
df_filter = df_test[df_test['lenght'] > 3]

re_list = df_filter['words'].to_list()
print(len(re_list))

272686


In [40]:
with open("text_final_final.txt") as file:
    content = file.read()

content = [word.strip() for word in content.split()]
dict_words = {}

for word in tqdm(content):
    if word not in list(dict_words.keys()):
        repetitions = content.count(word)
        rep_list = [word for i in range(repetitions)]
        dict_words[word] = rep_list
        
    else:
        pass


100%|██████████| 262252/262252 [00:35<00:00, 7425.55it/s]  


In [46]:
counter = 0
dict_final = {}

for k in sorted(dict_words, key=lambda k: len(dict_words[k]), reverse=True):
    dict_final[k] = dict_words[k]
    counter += 1
    
    if counter >= 150:
        break
        
print(list(dict_final.keys())[0], len(dict_final[list(dict_final.keys())[0]]))
list_final = []
last_lenght = len(dict_final[list(dict_final.keys())[-1]])

for key in dict_final.keys():
    list_final.extend(dict_final[key][0:len(dict_final[key])-last_lenght])
    
str_final = ' '.join(list_final)

text_file = open("esta si.txt", "w", encoding="utf-8")
text_file.write(str_final)
text_file.close()

print([word for word in list(dict_final.keys())[0:20]])

clases 6079
['clases', 'danza', 'para', 'arabe', 'dance', 'yoga', 'miguel', 'baile', 'danzas', 'ni', 'gran', 'santiago', 'salsa', 'como', 'arabes', 'academia', 'metro', 'avenida', 'ballet', 'cursos']


<h2>

In [39]:
final_list = []
counter = 0

for key in dict_words.keys():
    
    if len(dict_words[key]) > 100:
        final_list.extend(dict_words[key])
        counter += 1
    
    elif counter >= 150:
        break
        
print(len(final_list))

str_final = ' '.join(final_list)

text_file = open("text_final_final_final.txt", "w", encoding="utf-8")
text_file.write(str_final)
text_file.close()

print([word for word in ])

113420
done


In [200]:
final = []

for i, word in enumerate(re_list):
    try:
        re_word = re.search('([A-Za-z]+)', word).group(1)
        final.append(re_word)
        
    except:
        pass

print(len(final))
str_final = ' '.join(final)

text_file = open("text_final_final.txt", "w", encoding="utf-8")
text_file.write(str_final)
text_file.close()

print('done')

262252
done


In [155]:
from distributed import Client, LocalCluster, progress

In [156]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:62831 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 3 Cores: 6 Memory: 17.02 GB


In [171]:
import copy


def gen_chunks(combs, sample, size):
    if isinstance(combs, dict):
        combs = list(combs.keys())
        
    if sample == None:
        sampled_list = combs
    else:
        sampled_list = combs[sample[0]:sample[1]]

    for i in range(0, len(sampled_list), size):
        yield sampled_list[i:i + size]


def reduce_words(list_words, chunk, word, pos):
    returns = []
    
    for i, word in enumerate(chunk):
        local = copy.deepcopy(list_words)
        local.pop(pos + i)
        
        if word in local:
            returns.append(word)
        
        del local
        
    return returns
        

all_text = ''.join([value[1] for value in dict_texts.values()])
list_words = all_text.split(' ')

chunks = gen_chunks(list_words, [0, 1000], 100) #round(len(list_words)/6))

futures = [client.submit(reduce_words, list_words, chunk, word, pos) for pos, word in enumerate(list_words)]
results = client.gather(futures)

    
#final = ''.join(clean)

#print(len(final))
#text_file = open("texts.txt", "w", encoding="utf-8")
#text_file.write(all_text)
#text_file.close()

C:\Users\nicol\anaconda3\lib\site-packages\distributed\worker.py:3387: UserWarning: Large object of size 3.48 MB detected in task graph: 
  (['', '+56945218625', 'danzaarabe@bellydancercl',  ... iles?'], '', 0)
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
urls = ['https://nataliadufuur.com/', 'http://www.danaamar.com/', 'http://www.nurdeoriente.cl/', 'http://www.nayra.cl/', 'https://www.bellydancer.cl/', 'http://danzaarabechile.cl/', 'https://www.lovebellydanceonline.com/', 'https://www.safidanzaarabe.com/?fbclid=IwAR0h4x7enqP0LILu8vNccaSS1kQL_hDvbL-Lejj3vNbcZyP5230jbiwCjCc', 'https://danzadelvientretribal.cl/', '', 'http://www.seyyalbellydancer.com/', 'https://danielabellydancer.com/', 'http://natalia-abdallah.com/clases-de-danzas-arabes-online/?gclid=Cj0KCQiAhP2BBhDdARIsAJEzXlGmqQCPdOZG7e8rQLKgdyNNXWwuq-9EI8Ghvjq8Swgw1xJxl2eQlyIaAuHhEALw_wcB', 'https://orientaldanceonline.com/', 'https://www.danzaire.cl/clases-de-baile-en-santiago/clases-de-danza-arabe/', 'https://clasessanmiguel.blogspot.com/', '', 'http://buscadanza.cl/', 'https://www.artfitcenter.cl/danza-arabe/', 'https://www.arabesca.cl/?fbclid=IwAR2I1sQLi9yI1LaIeiG0u7a97Wls1Gaq4Ih8YPTVTyDRJzzBS4ZYkru0p_M', 'https://www.sandricastillo.com/', '', 'https://academiazamia.blogspot.com/?fbclid=IwAR2rONEJccxUky8RiXU2APmkCsficG4JEpTOizuKF5oSPHDvvhYNgtIcLtk']
dict_texts = {}

for url in tqdm(urls):
    try:
        basic_url = re.search('(.*\.com|.*\.cl)', url).group(1)
        dict_texts[basic_url] = get_text(basic_url)
        
        file_name = re.search('(https://www\.|http://www\.|https://|http://)(.*)\.', basic_url).group(2)
        
        text_file = open(f'{file_name}.txt', "w", encoding="utf-8")
        text_file.write(dict_texts[basic_url])
        text_file.close()
        
        print(file_name, basic_url)
        
    except Exception as e:
        print(url, ' failed by ', e)
        pass
    
all_texts = str(', '.join(list(dict_texts.values())))

text_file = open("texts.txt", "w", encoding="utf-8")
text_file.write(all_texts)
text_file.close()